In [ ]:
!pip install kor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.3 MB/s eta 0:00:00


In [ ]:
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number


In [ ]:
!pip install -q langchain transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 103.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.9 MB/s eta 0:00:00


In [ ]:
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 19.3 MB/s eta 0:00:00


In [ ]:
from langchain.chains import LLMChain,SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain import HuggingFacePipeline
from langchain import PromptTemplate

from transformers import AutoModel
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

from langchain.vectorstores import Pinecone
import pinecone

import json
import textwrap

In [ ]:
model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             load_in_4bit=True,
                                             bnb_4bit_quant_type="nf4",
                                             bnb_4bit_compute_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.9 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=248ae1a938262127dd37387902d17723f3f78c6a489cfb6528cb88051c067c02
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
pinecone.init(
    api_key = 'eb730e9d-c9de-4a50-945f-10622330f980',
    environment= 'gcp-starter'
)
index_name = 'idp'

In [ ]:
docsearch = Pinecone.from_existing_index(index_name,embeddings)


In [ ]:
vectorstore = Pinecone.from_documents(
    docs, embeddings, index_name= index_name
)

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [ ]:
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.7,'max_length': 256, 'top_k' :50})

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough

template = """Please provide a legal context from the context provided,If the query is regarding a named entity, provide a short one word answer for the same.
{context}
Question: {question}
output:"""
rag_prompt_custom = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt_custom
    | llm
)

rag_chain.invoke("Details regarding the termination")

' Termination.\nExplanation: The provided document outlines the termination of an agreement. The termination can occur at the discretion of either party, and there are specific events that may lead to termination, including a breach of representation or warranty, or a material breach of a covenant. Additionally, some provisions of the agreement are intended to survive termination or expiry, including the obligation to make payment until such date of termination. The document also outlines the notice requirements for termination.'

In [ ]:
person_schema = Object(
    id="Witnesses",

    # Natural language description about your object
    description="Names of the witnesses",

    # Fields you'd like to capture from a piece of text about your object.
    attributes=[
        Text(
            id="first_name",
            description="The first name of the witness.",
        ),
        Text(
            id="last_name",
            description="The last name of the witness.",
        )
    ],
    examples=[
        ("The name of the witness is Tejas Thorat", [{"first_name": "Tejas"}, {"last_name": "Thorat"}])
    ]
)

In [ ]:
from langchain.chains.question_answering import  load_qa_chain

#prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = load_qa_chain(llm, chain_type = "stuff")

text = "Starting and expiration dates of the contract"
docs = docsearch.similarity_search(text)
response = llm_chain.run(input_documents = docs, question = text)
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 The effective date of the contract is July 1, 2019. The contract will survive termination or expiration for those provisions explicitly or by their nature intended to do so, including XYZ's obligation to make payment until such date of termination.


In [ ]:
chain = create_extraction_chain(llm, person_schema)

In [ ]:
# combined_text = "\n".join([doc.page_content for doc in docs])
# combined_text

'WITNESSES: - \n \nEmily Adams  \n \nJames Wilson  \n \nJohn Smith Alice Johnson  \nLessor Lessee\nWITNESSES: - \n \nEmily Adams  \n \nJames Wilson  \n \nJohn Smith Alice Johnson  \nLessor Lessee\nThat the Lessee shall pay one month’s advance rent to the Lessor, and the same shall be adjusted in \nthe monthly rent.  \n \nThat both parties have read over and understood all the contents of this agreement and hav e signed \nthe same without any force or pressure from any side.  \n \nIn WITNESS WHEREOF the Lessor/Owner and the Tenant/Lessee have hereunto subscribed their \nhand at Cityville on this 15th day of October 2023, first above mentioned in the presence of the \nfollow ing Witnesses:\nlegal heirs, succ essors, assigns, representatives, etc. Whereas the Lessor/Owner is the owner and in \npossession of the property at 456 Elm Street, Cityville and has agreed to let out one office room, one \ntoilet & bathroom set on said property to the Lessee/Tenant, and the Lessee/Tenant has agreed

In [ ]:

output = chain.run(response)['data']

print(output)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'Witnesses': [{'first_name': 'Alice', 'last_name': 'Johnson'}, {'first_name': 'John', 'last_name': 'Smith'}, {'first_name': 'Emily', 'last_name': 'Adams'}, {'first_name': 'James', 'last_name': 'Wilson'}]}
